In [ ]:
import jax, flax, numpy
import flax.linen as nn
from notebookinit import *
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import optax
import tiktoken
import lib_kata
from tqdm.auto import tqdm2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22  8  3
 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8  0 14
  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7

In [ ]:
text_encoded, text_encoder, text_decoder, vocabulary_size = lib_kata.load_dataset_and_tokenize()

vocabulary: 
 ,e,t,o,a,i,h,s,r,n,
,l,d,u,m,y,w,,,c,f,g,b,p,:,k,v,.,',;,?,!,-,j,q,x,z,3,&,$ 
 size: 39 chars
first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [ ]:
context_size=64
embedd_features=7
learning_rate=3e-2
hidden_features_1 = 64
hidden_features_2 = 64

In [ ]:
X_all,y_all = lib_kata.make_training_Xy(text_encoded, context_size=context_size)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.05, shuffle=False, random_state=42)

In [ ]:
lib_kata.preview_Xy(X=X_train, y=y_train, text_decoder=text_decoder, count=5, offset=0)

[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11]  ->  11  |  first citizen:
before we proceed any further, hear me speak.

al _ l
[ 5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0
 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4
  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11]  ->  23  |  irst citizen:
before we proceed any further, hear me speak.

all _ :
[ 8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22
  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8
  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23]  ->  10  |  rst citizen:
before we proceed any further, hear me speak.

all: _ 

[ 7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22  8
  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8  0
 14  1  0  7 22  1  4 2

In [ ]:
vocabulary_size

39

In [ ]:
embedder = nn.Embed(num_embeddings=vocabulary_size, features=embedd_features)

In [ ]:
embedder

In [ ]:
embedder_example_param = embedder.init(jax.random.PRNGKey(0), X_train[0])

In [ ]:
example_X = X_train[0].reshape([1,context_size])
example_X.shape

(1, 64)

In [ ]:
tabulate_fn = nn.tabulate(embedder, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(example_X))

                               Embed Summary                                
┏━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path ┃ module ┃ inputs      ┃ outputs         ┃ params                   ┃
┡━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      │ Embed  │ int32[1,64] │ float32[1,64,5] │ embedding: float32[39,5] │
│      │        │             │                 │                          │
│      │        │             │                 │ 195 (780 B)              │
├──────┼────────┼─────────────┼─────────────────┼──────────────────────────┤
│      │        │             │           Total │ 195 (780 B)              │
└──────┴────────┴─────────────┴─────────────────┴──────────────────────────┘
                                                                            
                       Total Parameters: 195 (780 B)                        

In [ ]:
X_small,y_small = lib_kata.make_training_Xy(text_encoded[0:2000], context_size=context_size)

example_X=X_small[0].reshape([1,context_size])

example_target_y=y_small[0].reshape([-1,1])
example_target_y_logits = nn.activation.one_hot(example_target_y,vocabulary_size)
example_target_y_logits

Array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [ ]:
text_decoder(jnp.argmax(example_target_y_logits).reshape([-1,1]))

'l'

In [ ]:

    
class SimpleNN1(nn.Module):
    # batch size is implied from data
    context_size: int = 8    
    embedd_features: int = 5
    hidden_features_1: int = 16
    hidden_features_2: int = 16
    vocabulary_size: int = 39
    

    def setup(self):
        # print(self.context_size, self.embedd_features, "->", self.context_size * self.embedd_features,)
        pass
    
    @nn.compact
    def __call__(self, x):
        # first, embedd the input
        y0 = nn.Embed(name='embedding',num_embeddings=self.vocabulary_size, features=self.embedd_features)(x)
        # flatten the input vector, as there is no conceptual difference between the token's identity and token's feature here:        
        h = y0.reshape([-1,self.context_size*self.embedd_features])
        
        h = nn.Dense(name='layer_1',features=self.hidden_features_1)(h)
        h = nn.leaky_relu(h)        
        h = nn.Dense(name='layer_2',features=self.hidden_features_2)(h)
        h = nn.leaky_relu(h)        

        
        # final dense layer down to prediction logits. Note that there might be more logits than in the hidden state, that's OK.
        h = nn.Dense(name='layer_final',features=self.vocabulary_size)(h)                        
        # h = nn.activation.softmax(h)        
        y = h
        return y


simpleNN = SimpleNN1(context_size=context_size, embedd_features=embedd_features, hidden_features_1=hidden_features_1, hidden_features_2=hidden_features_2)
simpleNN_params = simpleNN.init(jax.random.PRNGKey(0), example_X)
tabulate_fn = nn.tabulate(simpleNN, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(example_X))
print(f'{simpleNN.apply(simpleNN_params,example_X).shape=}')
predicted_logits = simpleNN.apply(simpleNN_params,example_X)
predicted_example_token = jnp.argmax(simpleNN.apply(simpleNN_params,example_X), axis=1)
predicted_logits_str = ", ".join([f'{float(predicted_logits[1,x]):0.2f}' for x in range(predicted_logits.shape[1])])
print(predicted_logits_str)
print(f'predicted token: ', predicted_example_token, "-> >>>", text_decoder(predicted_example_token),"<<<")
print(f'   target token: ', example_target_y, f"confidence: {float(predicted_logits[0,predicted_example_token][0]):0.3f}")


                                    SimpleNN1 Summary                                    
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path        ┃ module    ┃ inputs         ┃ outputs         ┃ params                   ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             │ SimpleNN1 │ int32[1,64]    │ float32[1,39]   │                          │
├─────────────┼───────────┼────────────────┼─────────────────┼──────────────────────────┤
│ embedding   │ Embed     │ int32[1,64]    │ float32[1,64,5] │ embedding: float32[39,5] │
│             │           │                │                 │                          │
│             │           │                │                 │ 195 (780 B)              │
├─────────────┼───────────┼────────────────┼─────────────────┼──────────────────────────┤
│ layer_1     │ Dense     │ float32[1,320] │ float32[1,64]   │ bias: float32[64]        │
│             │           │                │                 │ kernel: float32[320,64]  │
│             │           │                │                 │                          │
│             │           │                │                 │ 20,544 (82.2 KB)         │
├─────────────┼───────────┼────────────────┼─────────────────┼──────────────────────────┤
│ layer_2     │ Dense     │ float32[1,64]  │ float32[1,64]   │ bias: float32[64]        │
│             │           │                │                 │ kernel: float32[64,64]   │
│             │           │                │                 │                          │
│             │           │                │                 │ 4,160 (16.6 KB)          │
├─────────────┼───────────┼────────────────┼─────────────────┼──────────────────────────┤
│ layer_final │ Dense     │ float32[1,64]  │ float32[1,39]   │ bias: float32[39]        │
│             │           │                │                 │ kernel: float32[64,39]   │
│             │           │                │                 │                          │
│             │           │                │                 │ 2,535 (10.1 KB)          │
├─────────────┼───────────┼────────────────┼─────────────────┼──────────────────────────┤
│             │           │                │           Total │ 27,434 (109.7 KB)        │
└─────────────┴───────────┴────────────────┴─────────────────┴──────────────────────────┘
                                                                                         
                           Total Parameters: 27,434 (109.7 KB)                           




simpleNN.apply(simpleNN_params,example_X).shape=(1, 39)
-0.07, -0.12, 0.07, -0.11, -0.11, -0.04, -0.15, 0.01, -0.05, 0.19, -0.11, 0.10, 0.12, -0.12, 0.15, -0.25, 0.11, -0.16, -0.18, -0.11, 0.06, 0.24, -0.02, -0.25, 0.01, 0.44, 0.12, -0.22, 0.11, -0.06, 0.19, 0.18, 0.12, 0.22, -0.04, -0.31, -0.13, 0.10, 0.10
predicted token:  [25] -> >>> v <<<
   target token:  [[11]] confidence: 0.441


In [ ]:
example_target_y

array([[11]], dtype=int32)

In [ ]:
# def cross_entropy_loss(logits, labels):
#     """Returns cross-entropy loss."""
#     assert jnp.all(logits.shape==labels.shape)
#     return -jnp.mean(jnp.sum(logits * labels, axis=1))

In [ ]:
predicted_logits

Array([[-0.07074139, -0.12344126,  0.07259806, -0.11130323, -0.10570294,
        -0.03674741, -0.14856309,  0.00942318, -0.04642118,  0.18656567,
        -0.11203478,  0.10441886,  0.11849858, -0.11622968,  0.14515294,
        -0.2509525 ,  0.10969354, -0.15764211, -0.1762435 , -0.10634437,
         0.05737808,  0.24254847, -0.01626321, -0.25410834,  0.00749659,
         0.4409358 ,  0.11811624, -0.21976818,  0.110365  , -0.06123686,
         0.18959042,  0.18318759,  0.12290651,  0.2231922 , -0.03789646,
        -0.30535108, -0.13487788,  0.09567506,  0.10127572]],      dtype=float32)

In [ ]:
predicted_logits.shape

(1, 39)

In [ ]:
example_target_y.shape

(1, 1)

In [ ]:
optax.softmax_cross_entropy_with_integer_labels(predicted_logits, example_target_y[0])

Array([3.5732367], dtype=float32)

In [ ]:
# example loss between the target and predicted
jnp.log10(optax.softmax_cross_entropy_with_integer_labels(predicted_logits, example_target_y[0]))

Array([0.5530618], dtype=float32)

In [ ]:
# I think I am ready to do a training step

In [ ]:
train_minibatch_size = 32*4096
batches_per_epoch = len(X_train) - train_minibatch_size
print(f'minibatches per epoch: {batches_per_epoch} with minibatch size of {train_minibatch_size}')
starter_X_minibatch = jnp.array(X_train[0:train_minibatch_size])
starter_y_minibatch = jnp.array(X_train[0:train_minibatch_size])

simpleNN_minibatched = SimpleNN1(context_size=context_size, embedd_features=embedd_features, hidden_features_1=hidden_features_1, hidden_features_2=hidden_features_2)
simpleNN_minibatched_params = simpleNN.init(jax.random.PRNGKey(0), starter_X_minibatch)
tabulate_fn = nn.tabulate(simpleNN_minibatched, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(starter_X_minibatch))

minibatches per epoch: 928491 with minibatch size of 131072


                                         SimpleNN1 Summary                                         
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path        ┃ module    ┃ inputs              ┃ outputs              ┃ params                   ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             │ SimpleNN1 │ int32[131072,64]    │ float32[131072,39]   │                          │
├─────────────┼───────────┼─────────────────────┼──────────────────────┼──────────────────────────┤
│ embedding   │ Embed     │ int32[131072,64]    │ float32[131072,64,5] │ embedding: float32[39,5] │
│             │           │                     │                      │                          │
│             │           │                     │                      │ 195 (780 B)              │
├─────────────┼───────────┼─────────────────────┼──────────────────────┼──────────────────────────┤
│ layer_1     │ Dense     │ float32[131072,320] │ float32[131072,64]   │ bias: float32[64]        │
│             │           │                     │                      │ kernel: float32[320,64]  │
│             │           │                     │                      │                          │
│             │           │                     │                      │ 20,544 (82.2 KB)         │
├─────────────┼───────────┼─────────────────────┼──────────────────────┼──────────────────────────┤
│ layer_2     │ Dense     │ float32[131072,64]  │ float32[131072,64]   │ bias: float32[64]        │
│             │           │                     │                      │ kernel: float32[64,64]   │
│             │           │                     │                      │                          │
│             │           │                     │                      │ 4,160 (16.6 KB)          │
├─────────────┼───────────┼─────────────────────┼──────────────────────┼──────────────────────────┤
│ layer_final │ Dense     │ float32[131072,64]  │ float32[131072,39]   │ bias: float32[39]        │
│             │           │                     │                      │ kernel: float32[64,39]   │
│             │           │                     │                      │                          │
│             │           │                     │                      │ 2,535 (10.1 KB)          │
├─────────────┼───────────┼─────────────────────┼──────────────────────┼──────────────────────────┤
│             │           │                     │                Total │ 27,434 (109.7 KB)        │
└─────────────┴───────────┴─────────────────────┴──────────────────────┴──────────────────────────┘
                                                                                                   
                                Total Parameters: 27,434 (109.7 KB)                                

In [ ]:
len(X_train)

1059563

In [ ]:
def prep_minibatch(minibatch_idx=0):
    minibatch_ptr = minibatch_idx
    X_minibatch = jnp.array(X_train[minibatch_ptr:minibatch_ptr+train_minibatch_size])
    y_minibatch = jnp.array(y_train[minibatch_ptr:minibatch_ptr+train_minibatch_size])
    y_minibatch_target_logits = nn.activation.one_hot(y_minibatch,vocabulary_size)
    return X_minibatch, y_minibatch, y_minibatch_target_logits
minibatch_idx = 4    

In [ ]:
minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)

In [ ]:
for idx_step in range(5):
    print(minibatch_X[idx_step,:], "->", minibatch_y[idx_step])

[ 2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22  8  3
 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8  0 14
  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7] -> 22
[ 0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22  8  3 18
  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8  0 14  1
  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22] -> 1
[18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22  8  3 18  1
  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8  0 14  1  0
  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1] -> 4
[ 5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22  8  3 18  1  1
 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8  0 14  1  0  7
 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4] -> 24
[ 2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0 22  8  3 18  1  1 12
  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4  8  0 14  1  0  7 22
  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24

In [ ]:
model_starter_params = simpleNN_minibatched_params
model_moving_params = model_starter_params.copy()

In [ ]:
optimizer = optax.adabelief(learning_rate=learning_rate)
optimizer_state = optimizer.init(model_moving_params)

In [ ]:
step_count=0
minibatch_idx = 0

In [ ]:
minibatch_idx

0

In [ ]:
# step
for idx_minibatch in tqdm(range(2000)):
    step_count+=1
    minibatch_idx = step_count % batches_per_epoch
    
    minibatch_X, minibatch_y, y_minibatch_target_logits =prep_minibatch(minibatch_idx=minibatch_idx)
    
    loss_fn = lambda model_params : jnp.mean(optax.softmax_cross_entropy_with_integer_labels(simpleNN_minibatched.apply(model_params,minibatch_X),  minibatch_y))
    loss_value, gradients = jax.value_and_grad(loss_fn)(model_moving_params)
    
    model_param_updates, optimizer_state = optimizer.update(gradients, optimizer_state, params=model_moving_params)
    model_moving_params = optax.apply_updates(model_moving_params, model_param_updates)
    if idx_minibatch % 100 == 0 :
        print(f'training loss mag: {numpy.log10(loss_value):0.3f}')

  0%|          | 0/2000 [00:00<?, ?it/s]

training loss mag: 0.569
training loss mag: 0.265
training loss mag: 0.232
training loss mag: 0.217
training loss mag: 0.208
training loss mag: 0.202
training loss mag: 0.195
training loss mag: 0.193
training loss mag: 0.187
training loss mag: 0.186
training loss mag: 0.184
training loss mag: 0.181
training loss mag: 0.183
training loss mag: 0.180
training loss mag: 0.181
training loss mag: 0.180
training loss mag: 0.177
training loss mag: 0.178
training loss mag: 0.178
training loss mag: 0.177


In [ ]:
# starter_prompt

In [ ]:
starter_prompt = jnp.array(X_train[0])

def predict_text(model_params, prompt_encoded, new_characters=250):
    prediction_decoded = text_decoder(prompt_encoded)
    print(f'{prediction_decoded}|||||')
    running_prompt = prompt_encoded.copy()
    # print(running_prompt)
    for char_idx in range(new_characters):
        predicted_logits = simpleNN_minibatched.apply(model_params, running_prompt)
        predicted_token = jnp.argmax(predicted_logits).reshape((1,1))
        predicted_char = text_decoder(predicted_token)
        prediction_decoded = f'{prediction_decoded}{predicted_char}'
        running_prompt = jnp.hstack([running_prompt[1:],predicted_token[:,0]])
        # print(running_prompt)
    print(prediction_decoded)    
predict_text(model_moving_params, starter_prompt)


In [ ]:
raise

In [ ]:
!git add .
!git commit -m 'savegame. now appears to train.'
